In [14]:
# Libraries
import random
import os
import json

class DataBase:
    def __init__(self, filename="student.data"):
        self.filename = filename 
        self.check_and_create_file()

    def check_and_create_file(self):
        print(f"Checking if '{self.filename}' exists...")
        if not os.path.exists(self.filename):
            with open(self.filename, 'w') as file:
                file.write("")  # Create an empty file
            print(f"File '{self.filename}' created.")
        else:
            print(f"File '{self.filename}' exists.")

    def write(self, data):
        try:
            with open(self.filename, 'w') as fileHandler:
                json.dump(data, fileHandler, indent=2)
                fileHandler.flush()
                os.fsync(fileHandler.fileno())
            # print(f"Data written to {self.filename}")
        except Exception as e:
            print(f"An error occurred while writing to the file: {e}")

    def read(self):
        try:
            if not os.path.exists(self.filename):
                print(f"File '{self.filename}' does not exist.")
                return {"students": {}, "used_ids": []}
            if os.stat(self.filename).st_size == 0:
                print(f"File '{self.filename}' is empty.")
                return {"students": {}, "used_ids": []}
            with open(self.filename, 'r') as fileHandler:
                print('file read')
                return json.load(fileHandler)
        except json.JSONDecodeError:
            print("Error: File contains invalid JSON.")
            return {"students": {}, "used_ids": []}
        except Exception as e:
            print(f"An error occurred: {e}")
            return {"students": {}, "used_ids": []}


In [3]:
# Libraries
import random
import os
import json

class SubjectClass():

    def Gen_SubjectID(self):
        
        number = random.randint(1,999)
        formatted_id = f"{number:03d}"
        return(formatted_id)
    
    def Gen_Results(self):
        # Generate Marks + Grade in same function
        mark = random.randint(25,100)
        if mark <= 49:
            grade = 'F'
        elif mark >= 50 and mark <= 64:
            grade = 'P'
        elif mark >= 65 and mark <= 74:
            grade = 'C'
        elif mark >= 75 and mark <= 84:
            grade = 'D'
        elif mark > 85:
            grade = 'HD'
        
        return mark, grade
    
class DataBase:
    def __init__(self, filename="student.data"):
        self.filename = filename 
        self.check_and_create_file()

    def check_and_create_file(self):
        print(f"Checking if '{self.filename}' exists...")
        if not os.path.exists(self.filename):
            with open(self.filename, 'w') as file:
                file.write("")  # Create an empty file
            print(f"File '{self.filename}' created.")
        else:
            print(f"File '{self.filename}' exists.")

    def write(self, data):
        try:
            with open(self.filename, 'w') as fileHandler:
                json.dump(data, fileHandler, indent=2)
                fileHandler.flush()
                os.fsync(fileHandler.fileno())
            # print(f"Data written to {self.filename}")
        except Exception as e:
            print(f"An error occurred while writing to the file: {e}")

    def read(self):
        try:
            if not os.path.exists(self.filename):
                print(f"File '{self.filename}' does not exist.")
                return {"students": {}, "used_ids": []}
            if os.stat(self.filename).st_size == 0:
                print(f"File '{self.filename}' is empty.")
                return {"students": {}, "used_ids": []}
            with open(self.filename, 'r') as fileHandler:
                return json.load(fileHandler)
        except json.JSONDecodeError:
            print("Error: File contains invalid JSON.")
            return {"students": {}, "used_ids": []}
        except Exception as e:
            print(f"An error occurred: {e}")
            return {"students": {}, "used_ids": []}

class Backend():
    "Needs 'db' DataBase object to perform get_count function"

    def __init__(self, email):
        self.filename = 'student.data'
        self.db = DataBase()
        self.students = self.db.read()
        self.student = email
        self.student_file = self.students['students'].get(self.student, {}) 
        
    @staticmethod
    def print_col(text, colour):
        colours = {
            'blue': '\033[96m',   # Blue
            'yellow': '\033[93m', # Yellow
            'red': '\033[91m',    # Red
            'end': '\033[0m',
        }
        colour_code = colours.get(colour, colours['end'])
        print(f"{colour_code}{text}{colours['end']}")

    @staticmethod
    def standard_user_input():
        Backend.print_col("Student Course Menu (c/e/r/s/x):", 'blue')
        user_input = input()
        return user_input

    def check_sub(self):
        data = self.students
        if 'subjects' in data['students'][self.student]:
            return True
        return False

    def get_count(self):
        print(self.students)
        self.students = self.db.read() # <>
        if self.check_sub():
            return len(self.students['students'][self.student]['subjects'])
        else:
            return 0

    @staticmethod
    def update_password():
        new_password = input("New Password:")
        confirm_password = input("Confirm Password:")

        while confirm_password != new_password:
            Backend.print_col("Password does not match - try again", "red")
            confirm_password = input("Confirm Password:")

    def show(self):
        self.students = self.db.read() # <>
        
        subject_count = self.get_count()
        if subject_count > 0:
            Backend.print_col(f"Showing {subject_count} subjects", "yellow")
            subjects = self.students['students'][self.student]['subjects']
            for i in range(len(subjects)):
                print(f"  Subject::{subjects[i]['subject']} -- mark = {subjects[i]['mark']} -- grade == {subjects[i]['grade']}")
            user_input = Backend.standard_user_input()
            return user_input
        else:
            Backend.print_col("No subjects currently enrolled.", "red")
            user_input = Backend.standard_user_input()
            return user_input
    
    def removal(self):
        self.students = self.db.read() # <>
        
        while True:
            subject = str(input("Remove Subject by ID: "))
            if subject.lower() == 'b':
                return  # Exit the function and return to the main menu immediately

            data = self.students
            
            # Check if the student exists
            if 'students' in data and self.student in data['students']:
                student_data = data['students'][self.student]

                # Check if the subject is listed
                subjects = student_data.get('subjects', [])
                subject_to_remove = next((s for s in subjects if s['subject'] == subject), None)

                if subject_to_remove:
                    Backend.print_col(f"Dropping Subject-{subject}", "yellow")
                    # Remove the subject
                    subjects.remove(subject_to_remove)
                    print(self.students)
                    # Write updated data back to the file
                    self.db.write(data)
                    self.students = self.db.read() # <>
                    print(self.students)
                    Backend.print_col(f"You are now enrolled in {len(subjects)} out of 4 subjects", "yellow")
                    return
                else:
                    Backend.print_col(f"Error: Subject-{subject} is not in subject list. Available subjects are:", "red")
                    for s in subjects:
                        print(f" - Subject: {s['subject']}")
                    Backend.print_col(f"Or press 'b' to go back to menu", "red")
            else:
                Backend.print_col("Error: Student data not found.", "red")
                return

    def enrollment(self):
        
        self.students = self.db.read()
        
        subject_count = Backend.get_count(self)
        if subject_count >= 4: # Error Handling
            Backend.print_col("Students are allowed to enrol in 4 subjects only", "red")
            user_input = Backend.standard_user_input()
        else:
            subject_id = SubjectClass.Gen_SubjectID(self)
            mark, grade = SubjectClass.Gen_Results(self)
            Backend.print_col(f"Enrolling in Subject-{subject_id}", "yellow")
            
            # Load student data
            data = self.students
            if 'subjects' not in data['students'][self.student]:
                data['students'][self.student]['subjects'] = []

            subjects = {'subject': subject_id, 'mark': mark, 'grade': grade}
            data['students'][self.student]['subjects'].append(subjects)
            # print(f"Updated data: {data}")
            # Write updated data
            self.db.write(data)
            # print("Data written to file.")
            subject_count = Backend.get_count(self)
            self.students = self.db.read()
            print(self.students)
            Backend.print_col(f"You are now enrolled in {subject_count} out of 4 subjects", "yellow")
            user_input = Backend.standard_user_input()

        return user_input

class StuCourseSys():

    def __init__(self, email):

        self.db = DataBase()
        self.be = Backend(email)
        self.students = self.db.read()
        self.user_input = self.be.standard_user_input()
        self.correct_inputs = ['x', 'c', 'e', 'r', 's']

    def main(self):

        while self.user_input != 'x':
            
            if self.user_input == 'c': # Changing Subject
                self.be.print_col("Updating Password", "yellow")
                self.be.update_password()
                self.user_input = self.be.standard_user_input()
                
            elif self.user_input == 'e': # Enrollment 
                self.user_input = self.be.enrollment()
                
                self.students = self.db.read() # updating ? 
                
            elif self.user_input == 'r':  # Removing Subject
                self.be.removal()
                self.bestudents = self.db.read() # updating ? 
                
                self.user_input = self.be.standard_user_input()
                    
            elif self.user_input == 's': # Showing subjects
                self.user_input = self.be.show()

            elif self.user_input not in self.correct_inputs: # Error Handling
                self.be.print_col(f"Input {self.user_input} not a valid input. Try again...", "red") 
                self.user_input = self.be.standard_user_input()

In [15]:
class Backend:
    def __init__(self, email):
        self.db = DataBase()
        self.students = self.db.read()
        self.student = email
        self.student_file = self.students['students'].get(self.student, {})

    @staticmethod
    def print_col(text, colour):
        colours = {
            'blue': '\033[96m',   # Blue
            'yellow': '\033[93m', # Yellow
            'red': '\033[91m',    # Red
            'end': '\033[0m',
        }
        colour_code = colours.get(colour, colours['end'])
        print(f"{colour_code}{text}{colours['end']}")

    @staticmethod
    def standard_user_input():
        Backend.print_col("Student Course Menu (c/e/r/s/x):", 'blue')
        user_input = input()
        return user_input

    def check_sub(self):
        self.students = self.db.read()  # Ensure latest data is read
        self.student_file = self.students['students'].get(self.student, {})
        return 'subjects' in self.student_file

    def get_count(self):
        self.students = self.db.read()  # Ensure latest data is read
        self.student_file = self.students['students'].get(self.student, {})
        if self.check_sub():
            return len(self.student_file['subjects'])
        return 0

    def show(self):
        
        self.students = self.db.read()  # Ensure latest data is read
        self.student_file = self.students['students'].get(self.student, {})
        subject_count = self.get_count()
        if subject_count > 0:
            Backend.print_col(f"Showing {subject_count} subjects", "yellow")
            subjects = self.student_file['subjects']
            for i in range(len(subjects)):
                print(f"  Subject::{subjects[i]['subject']} -- mark = {subjects[i]['mark']} -- grade == {subjects[i]['grade']}")
            self.user_input = Backend.standard_user_input()
            return self.user_input
        else:
            Backend.print_col("No subjects currently enrolled.", "red")
            self.user_input = Backend.standard_user_input()
            return self.user_input

    def removal(self):
        self.students = self.db.read()  # Ensure latest data is read
        self.student_file = self.students['students'].get(self.student, {})
        
        while True:
            subject = str(input("Remove Subject by ID: "))
            if subject.lower() == 'b':
                return  # Exit the function and return to the main menu immediately

            if 'students' in self.students and self.student in self.students['students']:
                subjects = self.student_file.get('subjects', [])
                subject_to_remove = next((s for s in subjects if s['subject'] == subject), None)

                if subject_to_remove:
                    Backend.print_col(f"Dropping Subject-{subject}", "yellow")
                    subjects.remove(subject_to_remove)
                    self.db.write(self.students)  # Write updated data back to the file
                    self.students = self.db.read()  # Re-read the updated data
                    self.student_file = self.students['students'].get(self.student, {})
                    Backend.print_col(f"You are now enrolled in {len(subjects)} out of 4 subjects", "yellow")
                    return
                else:
                    Backend.print_col(f"Error: Subject-{subject} is not in subject list. Available subjects are:", "red")
                    for s in subjects:
                        print(f" - Subject: {s['subject']}")
                    Backend.print_col(f"Or press 'b' to go back to menu", "red")
            else:
                Backend.print_col("Error: Student data not found.", "red")
                return

    def enrollment(self):
        self.students = self.db.read()  # Ensure latest data is read
        self.student_file = self.students['students'].get(self.student, {})

        subject_count = self.get_count()
        if subject_count >= 4:  # Error Handling
            Backend.print_col("Students are allowed to enrol in 4 subjects only", "red")
            self.user_input = self.standard_user_input()
        else:
            subject_id = SubjectClass.Gen_SubjectID(self)
            mark, grade = SubjectClass.Gen_Results(self)
            Backend.print_col(f"Enrolling in Subject-{subject_id}", "yellow")

            if 'subjects' not in self.student_file:
                self.student_file['subjects'] = []

            new_subject = {'subject': subject_id, 'mark': mark, 'grade': grade}
            self.student_file['subjects'].append(new_subject)
            self.students['students'][self.student] = self.student_file
            self.db.write(self.students)  # Write updated data back to the file

            subject_count = self.get_count()
            Backend.print_col(f"You are now enrolled in {subject_count} out of 4 subjects", "yellow")
            self.user_input = self.standard_user_input()
        return self.user_input

class StuCourseSys:
    def __init__(self, email):
        self.db = DataBase()
        self.be = Backend(email)
        self.contents = self.db.read()
        self.user_input = self.be.standard_user_input()
        self.correct_inputs = ['x', 'c', 'e', 'r', 's']

    def main(self):
        while self.user_input != 'x':
            if self.user_input == 'c':  # Changing Subject
                self.be.print_col("Updating Password", "yellow")
                self.be.update_password()
                self.user_input = self.be.standard_user_input()
                # have to address the data base issue here, no record of passwords 
                
            elif self.user_input == 'e':  # Enrollment
                self.user_input = self.be.enrollment()
                self.contents = self.db.read()  # Update contents after enrollment
            
            elif self.user_input == 'r':  # Removing Subject
                self.be.removal()
                self.contents = self.db.read()  # Update contents after removal
                self.user_input = self.be.standard_user_input()
            
            elif self.user_input == 's':  # Showing subjects
                self.user_input = self.be.show()
            
            elif self.user_input not in self.correct_inputs:  # Error Handling
                self.be.print_col(f"Input {self.user_input} not a valid input. Try again...", "red")
                self.user_input = self.be.standard_user_input()

In [18]:
class Backend:
    def __init__(self, email):
        self.db = DataBase()
        self.refresh_data()
        self.student = email
        self.student_file = self.students['students'].get(self.student, {})
        print(f"Initialized Backend for {self.student}: {self.student_file}")  # Debugging

    def refresh_data(self):
        self.students = self.db.read()
        self.student_file = self.students['students'].get(self.student, {})
        print(f"Data refreshed for {self.student}: {self.student_file}")  # Debugging

    @staticmethod
    def print_col(text, colour):
        colours = {
            'blue': '\033[96m',   # Blue
            'yellow': '\033[93m', # Yellow
            'red': '\033[91m',    # Red
            'end': '\033[0m',
        }
        colour_code = colours.get(colour, colours['end'])
        print(f"{colour_code}{text}{colours['end']}")

    @staticmethod
    def standard_user_input():
        Backend.print_col("Student Course Menu (c/e/r/s/x):", 'blue')
        user_input = input()
        return user_input

    def check_sub(self):
        self.refresh_data()  # Ensure latest data is read
        return 'subjects' in self.student_file

    def get_count(self):
        self.refresh_data()  # Ensure latest data is read
        if self.check_sub():
            count = len(self.student_file['subjects'])
            print(f"Subject count for {self.student}: {count}")  # Debugging
            return count
        return 0

    def show(self):
        self.refresh_data()  # Ensure latest data is read
        subject_count = self.get_count()
        if subject_count > 0:
            Backend.print_col(f"Showing {subject_count} subjects", "yellow")
            subjects = self.student_file['subjects']
            for i in range(len(subjects)):
                print(f"  Subject::{subjects[i]['subject']} -- mark = {subjects[i]['mark']} -- grade == {subjects[i]['grade']}")
            self.user_input = Backend.standard_user_input()
            return self.user_input
        else:
            Backend.print_col("No subjects currently enrolled.", "red")
            self.user_input = Backend.standard_user_input()
            return self.user_input

    def removal(self):
        self.refresh_data()  # Ensure latest data is read
        
        while True:
            subject = str(input("Remove Subject by ID: "))
            if subject.lower() == 'b':
                return  # Exit the function and return to the main menu immediately

            if 'students' in self.students and self.student in self.students['students']:
                subjects = self.student_file.get('subjects', [])
                subject_to_remove = next((s for s in subjects if s['subject'] == subject), None)

                if subject_to_remove:
                    Backend.print_col(f"Dropping Subject-{subject}", "yellow")
                    subjects.remove(subject_to_remove)
                    self.db.write(self.students)  # Write updated data back to the file
                    self.refresh_data()  # Re-read the updated data
                    Backend.print_col(f"You are now enrolled in {len(subjects)} out of 4 subjects", "yellow")
                    return
                else:
                    Backend.print_col(f"Error: Subject-{subject} is not in subject list. Available subjects are:", "red")
                    for s in subjects:
                        print(f" - Subject: {s['subject']}")
                    Backend.print_col(f"Or press 'b' to go back to menu", "red")
            else:
                Backend.print_col("Error: Student data not found.", "red")
                return

    def enrollment(self):
        self.refresh_data()  # Ensure latest data is read

        subject_count = self.get_count()
        if subject_count >= 4:  # Error Handling
            Backend.print_col("Students are allowed to enrol in 4 subjects only", "red")
            self.user_input = self.standard_user_input()
        else:
            subject_id = SubjectClass.Gen_SubjectID(self)
            mark, grade = SubjectClass.Gen_Results(self)
            Backend.print_col(f"Enrolling in Subject-{subject_id}", "yellow")

            if 'subjects' not in self.student_file:
                self.student_file['subjects'] = []

            new_subject = {'subject': subject_id, 'mark': mark, 'grade': grade}
            self.student_file['subjects'].append(new_subject)
            self.students['students'][self.student] = self.student_file
            self.db.write(self.students)  # Write updated data back to the file

            self.refresh_data()  # Re-read the updated data
            subject_count = self.get_count()
            Backend.print_col(f"You are now enrolled in {subject_count} out of 4 subjects", "yellow")
            self.user_input = self.standard_user_input()
        return self.user_input

class StuCourseSys:
    def __init__(self, email):
        self.db = DataBase()
        self.be = Backend(email)
        self.contents = self.db.read()
        self.user_input = self.be.standard_user_input()
        self.correct_inputs = ['x', 'c', 'e', 'r', 's']

    def main(self):
        while self.user_input != 'x':
            if self.user_input == 'c':  # Changing Subject
                self.be.print_col("Updating Password", "yellow")
                self.be.update_password()
                self.user_input = self.be.standard_user_input()
                # have to address the data base issue here, no record of passwords 
                
            elif self.user_input == 'e':  # Enrollment
                self.user_input = self.be.enrollment()
                self.contents = self.db.read()  # Update contents after enrollment
            
            elif self.user_input == 'r':  # Removing Subject
                self.be.removal()
                self.contents = self.db.read()  # Update contents after removal
                self.user_input = self.be.standard_user_input()
            
            elif self.user_input == 's':  # Showing subjects
                self.user_input = self.be.show()
            
            elif self.user_input not in self.correct_inputs:  # Error Handling
                self.be.print_col(f"Input {self.user_input} not a valid input. Try again...", "red")
                self.user_input = self.be.standard_user_input()

In [19]:
from student_controller import StudentController
import cop_Student_Course_System_copy
import re

def main():
    controller = StudentController()
    
    while True:
        command = input("Student System (l/r/x): ").strip().lower()

        if command == 'x':
            print("\033[93mThank you!\033[0m")  # Yellow color for "Thank you!"
            break
        
        elif command == 'l':
            print("\033[92mStudent Sign In\033[0m")
            email = input("Enter your email: ").strip()
            password = input("Enter your password: ").strip()

            if not re.match(r'^[A-Z][a-zA-Z]{5,}\d{3,}$', password) or not re.match(r'^([a-zA-Z]+)\.([a-zA-Z]+)@university\.com$', email):
                print("\033[91mIncorrect email or password format\033[0m")
            else:
                success, message = controller.login_student(email, password)
                if success:
                    print("\033[92mLogin successful!\033[0m")
                    course_sys = cop_Student_Course_System_copy.StuCourseSys(email) 
                    course_sys.main()
                else:
                    print(controller.load_students())
                    print("\033[91mStudent does not exist\033[0m")

        elif command == 'r':
            print("\033[92mStudent Sign Up\033[0m")
            email = input("Enter your email: ").strip()
            password = input("Enter your password: ").strip()

            if not re.match(r'^[A-Z][a-zA-Z]{5,}\d{3,}$', password) or not re.match(r'^([a-zA-Z]+)\.([a-zA-Z]+)@university\.com$', email):
                print("\033[91mIncorrect email or password format\033[0m")
            else:
                try:
                    firstname, lastname = email.split('@')[0].split('.')
                    success, message = controller.register_student(firstname, lastname, email, password)
                    if success:
                        print("\033[93m{}".format(message), "\033[0m")
                    else:
                        print("\033[91m{}".format(message), "\033[0m")
                except ValueError:
                    print("\033[91mInvalid email format. Please use firstname.lastname@university.com\033[0m")
        else:
            print("\033[91mInvalid command. Please enter 'lto login, 'r' to register, or 'x' to exit.\033[0m")

if __name__ == '__main__':
    main()

Student System (l/r/x):  l


Student Sign In


Enter your email:  john.smith@university.com
Enter your password:  Helloworld123


Login successful!
Checking if 'student.data' exists...
File 'student.data' exists.
Checking if 'student.data' exists...
File 'student.data' exists.
Student Course Menu (c/e/r/s/x):


 s


No subjects currently enrolled.
Student Course Menu (c/e/r/s/x):


 x
Student System (l/r/x):  x


Thank you!


In [20]:
be = Backend("john.smith@university.com")
be.show()

Checking if 'student.data' exists...
File 'student.data' exists.
file read


AttributeError: 'Backend' object has no attribute 'student'

In [11]:
if __name__ == "__main__":
    system = StuCourseSys('carm.page@university.com')
    system.main()

Checking if 'student.data' exists...
File 'student.data' exists.
Checking if 'student.data' exists...
File 'student.data' exists.
Student Course Menu (c/e/r/s/x):


 s


Showing 13 subjects
  Subject::114 -- mark = 100 -- grade == HD
  Subject::435 -- mark = 53 -- grade == P
  Subject::886 -- mark = 63 -- grade == P
  Subject::108 -- mark = 46 -- grade == F
  Subject::912 -- mark = 32 -- grade == F
  Subject::709 -- mark = 79 -- grade == D
  Subject::219 -- mark = 25 -- grade == F
  Subject::204 -- mark = 57 -- grade == P
  Subject::652 -- mark = 87 -- grade == HD
  Subject::613 -- mark = 39 -- grade == F
  Subject::454 -- mark = 38 -- grade == F
  Subject::185 -- mark = 90 -- grade == HD
  Subject::061 -- mark = 73 -- grade == C
Student Course Menu (c/e/r/s/x):


 x
